In [1]:
# Create Model Using ResNet-50
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def resnet_block(inputs, filters, strides=1):
    x = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    
    if strides > 1:
        shortcut = Conv2D(filters, kernel_size=1, strides=strides, padding='same')(inputs)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = inputs
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    x = Conv2D(64, kernel_size=3, strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = resnet_block(x, filters=64)
    x = resnet_block(x, filters=64)
    
    x = resnet_block(x, filters=128, strides=2)
    x = resnet_block(x, filters=128)
    
    x = resnet_block(x, filters=256, strides=2)
    x = resnet_block(x, filters=256)
    x = resnet_block(x, filters=256)
    
    x = resnet_block(x, filters=512, strides=2)
    x = resnet_block(x, filters=512)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model

# Path ke folder dataset
dataset_path = '../Datasets'

# Parameter untuk model dan pelatihan
input_shape = (64, 64, 3)
num_classes = 12
batch_size = 64
epochs = 10

# Memuat dataset
train_data_dir = os.path.join(dataset_path + '/Train')
val_data_dir = os.path.join(dataset_path + '/Validation')
test_data_dir = os.path.join(dataset_path + '/Test')

# Mempersiapkan augmentasi gambar untuk data pelatihan
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Mempersiapkan generator data pelatihan
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

# Mempersiapkan augmentasi gambar untuk data validasi dan pengujian
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Mempersiapkan generator data validasi dan pengujian
val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

# Membuat model ResNet
model = create_resnet(input_shape, num_classes)

# Menentukan fungsi optimasi, fungsi loss, dan metrik evaluasi
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Melatih model dengan data generator
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size)

# Evaluasi model pada data pengujian
test_loss, test_acc = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size)

print('Test accuracy:', test_acc)

Found 19351 images belonging to 12 classes.
Found 4158 images belonging to 12 classes.
Found 4142 images belonging to 12 classes.
Epoch 1/10
302/302 [==============================] - 2219s 7s/step - loss: 0.9046 - accuracy: 0.6958 - val_loss: 5.9260 - val_accuracy: 0.1565
Epoch 2/10
302/302 [==============================] - 1901s 6s/step - loss: 0.5429 - accuracy: 0.8101 - val_loss: 0.8825 - val_accuracy: 0.7251
Epoch 3/10
302/302 [==============================] - 1893s 6s/step - loss: 0.4240 - accuracy: 0.8524 - val_loss: 0.8713 - val_accuracy: 0.7146
Epoch 4/10
302/302 [==============================] - 1899s 6s/step - loss: 0.3438 - accuracy: 0.8779 - val_loss: 0.6869 - val_accuracy: 0.7773
Epoch 5/10
302/302 [==============================] - 1887s 6s/step - loss: 0.3101 - accuracy: 0.8895 - val_loss: 0.4807 - val_accuracy: 0.8406
Epoch 6/10
302/302 [==============================] - 1777s 6s/step - loss: 0.2744 - accuracy: 0.9025 - val_loss: 0.7394 - val_accuracy: 0.7490
Epoch 

In [5]:
# Save model in format HDF5
h5_model_path = '../Model/model-frucheck.h5'

model.save(h5_model_path)

In [3]:
# Class Name
class_names = list(train_generator.class_indices.keys())
print(class_names)

['FreshApples', 'FreshBanana', 'FreshCarrot', 'FreshOranges', 'FreshPotato', 'FreshTomato', 'RottenApples', 'RottenBanana', 'RottenCarrot', 'RottenOranges', 'RottenPotato', 'RottenTomato']


In [4]:
### BATCH IMAGE PREDICTION ###

from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Load model
model_path = "../Model/model-frucheck.h5"
loaded_model = load_model(model_path)

# Define class names
class_names = ['ApelSegar', 'PisangSegar', 'WortelSegar', 'JerukSegar', 'KentangSegar', 'TomatSegar', 'ApelBusuk', 'PisangBusuk', 'WortelBusuk', 'JerukBusuk', 'KentangBusuk', 'TomatBusuk']

# Evaluate predictions for each class
dataset_path = '../Datasets'
test_data_dir = os.path.join(dataset_path + '/Test')
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Perform predictions for each batch of the test generator
predictions = loaded_model.predict_generator(test_generator, steps=len(test_generator))

# Loop through each class
for i, class_name in enumerate(class_names):
    class_indices = np.where(test_generator.classes == i)[0]
    class_images = np.array(test_generator.filenames)[class_indices]
    
    # Loop through each image in the class
    for image_path in class_images:
        image = Image.open(os.path.join(test_data_dir, image_path))
        
        # Perform prediction for the image
        image_array = np.array(image.resize((64, 64))) / 255.0
        input_image = np.expand_dims(image_array, axis=0)
        predicted_class = np.argmax(predictions[class_indices[class_images == image_path]])
        predicted_label = class_names[predicted_class]
        
        # VARIABLE HASIL KLASIFIKASI
        label_predicted = predicted_label.replace('Segar', '').replace('Busuk', '') # Klasifikasi nama buah/sayur
        freshness = 'Segar' if 'Segar' in predicted_label else 'Busuk' # Menentukan segar atau busuk
        
        # Calculate prediction accuracy
        predicted_prob = np.max(predictions[class_indices[class_images == image_path]])
        accuracy = predicted_prob * 100 # Akurasi prediksi
        
        # Print the predicted class, freshness, and accuracy
        print('Image:', image_path)
        print('Predicted Class:', label_predicted)
        print('Freshness:', freshness)
        print('Accuracy: {:.2f}%'.format(accuracy))
        print('---')

Found 4142 images belonging to 12 classes.


C:\Users\muham\AppData\Local\Temp\ipykernel_2516\233706549.py:30: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = loaded_model.predict_generator(test_generator, steps=len(test_generator))


Image: FreshApples\Apple_Fresh-100.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshApples\Apple_Fresh-102_augmented_2.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshApples\Apple_Fresh-104.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 97.14%
---
Image: FreshApples\Apple_Fresh-104_augmented_1.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 98.41%
---
Image: FreshApples\Apple_Fresh-109.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.94%
---
Image: FreshApples\Apple_Fresh-109_augmented_0.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.93%
---
Image: FreshApples\Apple_Fresh-109_augmented_1.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.83%
---
Image: FreshApples\Apple_Fresh-10_augmented_2.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.37%
---
Image: FreshApples\Apple_Fresh-11.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 97.37%
---
Image: FreshApples\Apple_Fresh-110_augmented_0.jp

Image: FreshApples\Apple_Fresh-229.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshApples\Apple_Fresh-229_augmented_2.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.99%
---
Image: FreshApples\Apple_Fresh-230.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.97%
---
Image: FreshApples\Apple_Fresh-230_augmented_3.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.94%
---
Image: FreshApples\Apple_Fresh-231_augmented_3.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshApples\Apple_Fresh-235_augmented_1.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshApples\Apple_Fresh-235_augmented_3.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshApples\Apple_Fresh-237.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshApples\Apple_Fresh-237_augmented_2.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshApples\Apple_Fresh-

Image: FreshApples\Apple_Fresh-376_augmented_2.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.89%
---
Image: FreshApples\Apple_Fresh-377.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.83%
---
Image: FreshApples\Apple_Fresh-378_augmented_0.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.84%
---
Image: FreshApples\Apple_Fresh-378_augmented_2.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.83%
---
Image: FreshApples\Apple_Fresh-379_augmented_2.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 100.00%
---
Image: FreshApples\Apple_Fresh-380_augmented_0.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.83%
---
Image: FreshApples\Apple_Fresh-380_augmented_2.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.74%
---
Image: FreshApples\Apple_Fresh-381_augmented_2.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 98.19%
---
Image: FreshApples\Apple_Fresh-386.jpg
Predicted Class: Apel
Freshness: Segar
Accuracy: 99.97%
---
Image: FreshApples\Apple

...

Image: RottenPotato\Potato_Rotten-227.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 94.08%
---
Image: RottenPotato\Potato_Rotten-228_augmented_3.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 98.72%
---
Image: RottenPotato\Potato_Rotten-229_augmented_1.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 86.07%
---
Image: RottenPotato\Potato_Rotten-231.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 70.79%
---
Image: RottenPotato\Potato_Rotten-232.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 94.21%
---
Image: RottenPotato\Potato_Rotten-232_augmented_0.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 98.08%
---
Image: RottenPotato\Potato_Rotten-233_augmented_0.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 94.45%
---
Image: RottenPotato\Potato_Rotten-235.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 69.77%
---
Image: RottenPotato\Potato_Rotten-235_augmented_0.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 90.04%
-

Image: RottenPotato\Potato_Rotten-389_augmented_2.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 97.05%
---
Image: RottenPotato\Potato_Rotten-391_augmented_2.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 87.67%
---
Image: RottenPotato\Potato_Rotten-393_augmented_2.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 95.74%
---
Image: RottenPotato\Potato_Rotten-394_augmented_2.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 72.90%
---
Image: RottenPotato\Potato_Rotten-395_augmented_0.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 92.31%
---
Image: RottenPotato\Potato_Rotten-395_augmented_2.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 70.76%
---
Image: RottenPotato\Potato_Rotten-397_augmented_1.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 96.08%
---
Image: RottenPotato\Potato_Rotten-398_augmented_0.jpg
Predicted Class: Kentang
Freshness: Busuk
Accuracy: 68.62%
---
Image: RottenPotato\Potato_Rotten-398_augmented_1.jpg
Predicted C

Image: RottenTomato\Tomato_Rotten-168_augmented_0.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 100.00%
---
Image: RottenTomato\Tomato_Rotten-170_augmented_2.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 93.58%
---
Image: RottenTomato\Tomato_Rotten-173_augmented_1.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 54.07%
---
Image: RottenTomato\Tomato_Rotten-174_augmented_0.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 84.28%
---
Image: RottenTomato\Tomato_Rotten-174_augmented_1.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 93.84%
---
Image: RottenTomato\Tomato_Rotten-176.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 57.11%
---
Image: RottenTomato\Tomato_Rotten-176_augmented_0.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 73.23%
---
Image: RottenTomato\Tomato_Rotten-176_augmented_2.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 69.91%
---
Image: RottenTomato\Tomato_Rotten-177_augmented_1.jpg
Predicted Class: Tomat
Freshness: Bus

Image: RottenTomato\Tomato_Rotten-349.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.86%
---
Image: RottenTomato\Tomato_Rotten-349_augmented_1.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.86%
---
Image: RottenTomato\Tomato_Rotten-350_augmented_0.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.96%
---
Image: RottenTomato\Tomato_Rotten-351_augmented_1.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.97%
---
Image: RottenTomato\Tomato_Rotten-351_augmented_2.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.98%
---
Image: RottenTomato\Tomato_Rotten-353_augmented_2.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.81%
---
Image: RottenTomato\Tomato_Rotten-355_augmented_2.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.52%
---
Image: RottenTomato\Tomato_Rotten-355_augmented_3.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.83%
---
Image: RottenTomato\Tomato_Rotten-358.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 

Image: RottenTomato\Tomato_Rotten-67.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 97.27%
---
Image: RottenTomato\Tomato_Rotten-6_augmented_1.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.84%
---
Image: RottenTomato\Tomato_Rotten-6_augmented_2.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.85%
---
Image: RottenTomato\Tomato_Rotten-70_augmented_2.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 98.53%
---
Image: RottenTomato\Tomato_Rotten-72.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 88.37%
---
Image: RottenTomato\Tomato_Rotten-73.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.96%
---
Image: RottenTomato\Tomato_Rotten-73_augmented_1.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 99.90%
---
Image: RottenTomato\Tomato_Rotten-74_augmented_0.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 68.75%
---
Image: RottenTomato\Tomato_Rotten-74_augmented_3.jpg
Predicted Class: Tomat
Freshness: Busuk
Accuracy: 75.64%
---
Image: Rotte

In [8]:
### SINGLE IMAGE PREDICTION ###

from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

# Load model
model_path = "../Model/model-frucheck.h5"
loaded_model = load_model(model_path)

# Define class names
class_names = ['ApelSegar', 'PisangSegar', 'WortelSegar', 'JerukSegar', 'KentangSegar', 'TomatSegar', 'ApelBusuk', 'PisangBusuk', 'WortelBusuk', 'JerukBusuk', 'KentangBusuk', 'TomatBusuk']

# Load and preprocess the image
image_path = '../Datasets/Test/RottenCarrot/Carrot_Rotten-41_augmented_3.jpg'  # SOURCE IMAGE
image = Image.open(image_path)
image = image.resize((64, 64))
image_array = np.array(image) / 255.0
input_image = np.expand_dims(image_array, axis=0)

# Perform prediction
predictions = loaded_model.predict(input_image)
predicted_class = np.argmax(predictions[0])
predicted_label = class_names[predicted_class]

# VARIABLE HASIL KLASIFIKASI
label_predicted = predicted_label.replace('Segar', '').replace('Busuk', '') # Klasifikasi nama buah/sayur
freshness = 'Segar' if 'Segar' in predicted_label else 'Busuk' # Menentukan segar atau busuk

# Calculate prediction accuracy
predicted_prob = np.max(predictions[0])
accuracy = predicted_prob * 100

# Print the predicted class, freshness, and accuracy
print('Image:', image_path)
print('Predicted Class:', label_predicted)
print('Freshness:', freshness)
print('Accuracy: {:.2f}%'.format(accuracy))

1/1 [==============================] - 1s 669ms/step
Image: ../Datasets/Test/RottenCarrot/Carrot_Rotten-41_augmented_3.jpg
Predicted Class: Wortel
Freshness: Busuk
Accuracy: 86.91%
